In [1]:
import json
import ssl

from annoy import AnnoyIndex
import cohere
import numpy as np
import pandas as pd

ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
with open('../keys.json', 'r') as f:
    api_key = json.load(f)['trial_api_key']
    
co = cohere.Client(api_key)

In [3]:
df = pd.read_csv(
    'https://storage.googleapis.com/cohere-assets/blog/text-clustering/'
    'data/askhn3k_df.csv',
    index_col=0)
print(df.shape)
df.head()

(3000, 14)


,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,"I'm a software engineer going blind, how shoul...",NaN,I&#x27;m a 24 y&#x2F;o full stack engineer (I ...,NaN,zachrip,3270,1587332026,2020-04-19 21:33:46+00:00,story,22918980,NaN,473.0,NaN,NaN
1,Am I the longest-serving programmer – 57 years...,NaN,"In May of 1963, I started my first full-time j...",NaN,genedangelo,2634,1590890024,2020-05-31 01:53:44+00:00,story,23366546,NaN,531.0,NaN,NaN
2,Is S3 down?,NaN,I&#x27;m getting<p>{\n &quot;errorCode&quot; ...,NaN,iamdeedubs,2589,1488303958,2017-02-28 17:45:58+00:00,story,13755673,NaN,1055.0,NaN,NaN
3,What tech job would let me get away with the l...,NaN,"Hey HN,<p>I&#x27;ll probably get a lot of flak...",NaN,lmueongoqx,2022,1617784863,2021-04-07 08:41:03+00:00,story,26721951,NaN,1091.0,NaN,NaN
4,What books changed the way you think about alm...,NaN,I was reflecting today about how often I think...,NaN,anderspitman,2009,1549387905,2019-02-05 17:31:45+00:00,story,19087418,NaN,1165.0,NaN,NaN


In [4]:
BATCH = 90
emb_list = []
n = len(df)

for i in range(0, n, BATCH):
    batch = df[i:min(i + BATCH, n)]
    txts = list(batch.title)
    embs_batch = co.embed(
        texts=txts,
        model='embed-english-v3.0',
        input_type='search_document'
    ).embeddings
    emb_list.extend(embs_batch)
    
emb = np.array(emb_list)
emb.shape

(3000, 1024)

In [5]:
search_idx = AnnoyIndex(emb.shape[1], 'angular')
for i, em in enumerate(emb):
    search_idx.add_item(i, em)
search_idx.build(10)  # 10 trees
search_idx.save('askhn.ann')

True

### 1 Given post title, retrieve most similar
(use nearest-neighbor search on embeddings)

In [9]:
ex_id = 47
similar_item_ids = search_idx.get_nns_by_item(
    ex_id, 10, include_distances=True)
res = pd.DataFrame({
    'post_titles': df.iloc[similar_item_ids[0]].title,
    'distance': similar_item_ids[1]
}).drop(ex_id)
print(f'Query post: "{df.iloc[ex_id].title}"\nNearest neigbors:')
res

Query post: "Urgent connection to Twitter support"
Nearest neigbors:


,post_titles,distance
2817,Twitter account stolen by presumed vulnerability,0.877691
1324,Desperate Python Hacker Seeking Help and Sugge...,0.937701
2808,LinkedIn suggesting I connect with my infertil...,0.946785
1488,What Has Happened to Twitter?,0.963125
168,"Help, HN'er in trouble, we need a laywer in De...",0.973216
346,Anonymous person sent proof of SSH access to o...,0.981433
2896,Alternate Email hosting to G Suite,0.988487
916,Website go-live checklist app,0.993343
2585,Google Drive/Office 365 alternatives,0.997686


### 2 Given a query we write, retrieve the most similar posts